#Pandas: Gerenciamento de dados de funcionários

Projeto de gerenciamento de dados de funcionários de uma empresa usando os conhecimentos aprendidos com Pandas.

Employee Data Management Project using Pandas

## Imports

In [1]:
#Importações de bibliotecas
#Imports
import pandas as pd
import datetime
import pytz

## Get data and create a DataFrame

In [ ]:
#Obter dados
#Get data
df_employees = pd.read_csv("employees.csv")
df_employees

In [ ]:
#Obter dados
#Get data
df_non_active_employees = pd.read_csv("non_active_employees.csv")
df_non_active_employees

## Backups

In [4]:
#Fazer backup do arquivo
#Create a backup of the file

#O código obtem a dia e hora e baixa o arquivo com esse tempo escrito no nome do arquivo.
#This code gets the day and time and downloads the file with the time written in the file name.
fuso_horario = pytz.timezone("America/Sao_Paulo")
time = datetime.datetime.now(fuso_horario)
df_employees.to_csv(f"employees_backup_{time.strftime('%d-%m-%Y_%H-%M-%S')}.csv", index=False)
df_non_active_employees.to_csv(f"non_active_employees_backup_{time.strftime('%d-%m-%Y_%H-%M-%S')}.csv", index=False)

In [ ]:
#Restaurar de backup employees.csv
#Restore

#Import os para verificar a existência do arquivo / Import os to check if the file exists
import os

fileName = input("Digite o nome do arquivo que deseja restaurar (com extensão): ")

if not os.path.exists(fileName):
  print("Arquivo não encontrado")
  exit()
else:
  df_employees_backup = pd.read_csv(fileName)
  df_employees_backup.to_csv("employees.csv", index=False)
  print("Backup restaurado com sucesso")
  df_employees = pd.read_csv("employees.csv")

In [ ]:
#Restaurar de backup non_active_employees.csv
#Restore

#Import os para verificar a existência do arquivo / Import os to check if the file exists
import os

fileName = input("Digite o nome do arquivo que deseja restaurar (com extensão): ")
if not os.path.exists(fileName):
  print("Arquivo não encontrado")
  exit()
else:
  df_non_active_employees_backup = pd.read_csv(fileName)
  df_non_active_employees_backup.to_csv("non_active_employees.csv", index=False)
  print("Backup restaurado com sucesso")
  df_non_active_employees = pd.read_csv("non_active_employees.csv")

## Data Manipulation

### Search data

In [ ]:
#Consultar dados do funcionário pesquisando pelo nome
#Search employee data searching by name

def search_employee_by_name(name):
  df_result = df_employees[df_employees["Nome"].str.contains(name)]

  if df_result.empty:
    print("Funcionário não encontrado")
    return

  return df_result

nameSearch = input("Digite o nome do funcionário: ")
search_employee_by_name(nameSearch)

In [ ]:
#Consultar dados do funcionário pesquisando pelo ID
#Search employee data searching by ID

def search_employee_by_id(id):
  df_result = df_employees[df_employees["ID"] == id]

  if df_result.empty:
    print("Funcionário não encontrado")
    return

  return df_result

id = int(input("Digite o id do funcionário: "))
search_employee_by_id(id)

### Add data

In [ ]:
#Adicionar novos funcionários
#Add new employees

#Check if ID have already been added
def check_id(id):
  while id in df_non_active_employees["ID"].values:
    id = id + 1
  return id

def add_new_employee(new_employee):
  #Create a new DataFrame and concat df_employees and new_employee
  df_new_employee = pd.DataFrame(new_employee, index=[0])

  df_employees_updated = pd.concat([get_main_df(), df_new_employee], ignore_index=True)
  df_employees_updated.to_csv("employees.csv", index=False)
  print("Funcionário adicionado com sucesso")

def get_new_employee_data():
  #Type employee data
  id = df_employees["ID"].max() + 1
  id = check_id(id)
  name = input("Digite o nome do novo funcionário: ")
  age = int(input("Digite a idade do novo funcionário: "))
  telephone = input("Digite o telefone do novo funcionário: ")
  email = input("Digite o email do novo funcionário: ")
  department = input("Digite o departamento do novo funcionário: ")
  position = input("Digite o cargo do novo funcionário: ")
  salary = float(input("Digite o salário do novo funcionário: "))
  date = time.strftime("%d/%m/%Y")

  new_employee = {"ID": id,
                  "Nome": name,
                  "Idade": age,
                  "Telefone": telephone,
                  "Email": email,
                  "Data de admissão": date,
                  "Departamento": department,
                  "Cargo": position,
                  "Salário": salary
                  }
  return new_employee

def get_main_df():
  return df_employees

new_employee = get_new_employee_data()
add_new_employee(new_employee)

#The main DataFrame read the updated file
df_employees = pd.read_csv("employees.csv")
df_employees

### Delete data

In [ ]:
#Excluir um funcionário
#Delete an employee

def delete_employee_by_id(id):
  df_result = df_employees[df_employees["ID"] == id]

  if df_result.empty:
    print("Funcionário não encontrado")
    return

  add_employee_in_non_active_employees(df_result)
  df_employees.drop(df_result.index, inplace=True)
  df_employees.reset_index(drop=True, inplace=True)
  print("Funcionário excluído com sucesso")

def delete_employee_by_name(name):
  df_result = df_employees[df_employees["Nome"].str.contains(name)]

  if df_result.empty:
    print("Funcionário não encontrado")
    return

  if len(df_result) > 1:
    df_result
    print("Encontramos mais de um funcionário com esse nome. Por favor, digite o ID do funcionário que deseja excluir.")
    id  = int(input("Digite o ID do funcionário que deseja excluir: "))
    delete_employee_by_id(id)
    return

  add_employee_in_non_active_employees(df_result)
  df_employees.drop(df_result.index, inplace=True)
  df_employees.reset_index(drop=True, inplace=True)
  print("Funcionário excluído com sucesso")



#Atualiza o DataFrame de Funcionários não ativos
#Update Non Active Employees DataFrame
def add_employee_in_non_active_employees(df_result):
  df_result = pd.DataFrame(df_result)
  df_result["Data de saída"] = time.strftime("%d/%m/%Y")

  df_non_active_employees_updated = pd.concat([get_non_active_employees_df(), df_result], ignore_index=True)
  df_non_active_employees_updated = df_non_active_employees_updated.sort_values(by="ID").reset_index(drop=True)
  df_non_active_employees_updated.to_csv("non_active_employees.csv", index=False)

def get_non_active_employees_df():
  return df_non_active_employees



#O usuário escolhe se vai excluir por ID ou pelo nome
#The user chooses whether to delete the employee by ID or by name
choose = input("Digite 1 para pesquisar por ID ou 2 para pesquisar por nome: ")
if(choose == "1"):
  id = int(input("Digite o ID do funcionário que deseja excluir: "))
  delete_employee_by_id(id)
elif(choose == "2"):
  name = input("Digite o nome do funcionário que deseja excluir: ")
  delete_employee_by_name(name)
else:
  print("Opção inválida")
  exit()



#Atualiza o DataFrame principal
#Update the main DataFrame
df_employees.to_csv("employees.csv")
df_non_active_employees = pd.read_csv("non_active_employees.csv")
df_employees

In [ ]:
#Check changes in non_active_employees DataFrame
df_non_active_employees

### Change data

In [ ]:
#Alterar dados de um funcionário
#Change employee data
def get_employee_by_id(id):
  df_result = df_employees[df_employees["ID"] == id]

  if df_result.empty:
    print("Funcionário não encontrado")
    return

  print("Dados do funcionário:")
  print(df_result)

  change_employee_data(df_result)

def get_employee_by_name():
  df_employees

def change_employee_data(df_result):
  choose_column = input("Digite o nome da coluna que deseja alterar: ")
  if choose_column not in df_result.columns:
    print("Coluna não encontrada")
    return

  employeeName = df_result["Nome"].values[0]
  new_value = input(f"Digite o novo dado para o funcionário {employeeName} na coluna {choose_column}: ")
  if choose_column == "Salário":
    new_value = float(new_value)
  if choose_column == "ID" or choose_column == "Idade":
    new_value = int(new_value)

  df_employees.loc[df_result.index, choose_column] = new_value

id = int(input("Digite o ID do funcionário que deseja alterar os dados: "))
get_employee_by_id(id)

#name = input("Digite o nome do funcionário que deseja alterar os dados: ")
#get_employee_by_name(name)

df_employees.to_csv("employees.csv")
df_employees

### Create new column

In [ ]:
#Criando uma coluna de Salário Anual
#Creating an Annual Salary column
df_employees["Salário Anual"] = df_employees["Salário"] * 12
df_employees.to_csv("employees.csv")
df_employees

### Separate data

In [ ]:
#Separar funcionários por departamento
#Separate employees by department

def get_employees_by_department(department):
  df_result = df_employees[df_employees["Departamento"] == department]

  if df_result.empty:
    print("Departamento não encontrado")
    return

  return df_result

department = input("Digite o departamento que deseja ver os funcionários: ")
get_employees_by_department(department)

## New DataFrames

### Departments

In [ ]:
#Dados gerais por departamento
#General data by departments

#Cria um array de departamentos
#Create an array called departments.
departments = df_employees["Departamento"].unique()


#Obtem a quantidade de pessoas por departamento
#Get the number of people by department
people_by_department = df_employees["Departamento"].value_counts()


#Obtem a media de idade de pessoas no departamento
#Get the average age of people in each department
average_age_by_department = df_employees.groupby("Departamento")["Idade"].mean().round(2)


#Obtem os principais dados relacionados a salario
#Get main data about salary
max_salary_by_department = df_employees.groupby("Departamento")["Salário"].max()
min_salary_by_department = df_employees.groupby("Departamento")["Salário"].min()
sum_salary_by_department = df_employees.groupby("Departamento")["Salário"].sum().round(2)
average_salary_by_department = df_employees.groupby("Departamento")["Salário"].mean().round(2)
annual_sum_salary_by_department = df_employees.groupby("Departamento")["Salário Anual"].sum().round(2)
annual_average_salary_by_department = df_employees.groupby("Departamento")["Salário Anual"].mean().round(2)


#Cria um DataFrame contendo todos os dados
#Create a DataFrame with all data
df_departments = pd.DataFrame(departments, columns=["Departamento"])
df_departments["Quantidade de Funcionários"] = df_departments["Departamento"].map(people_by_department)
df_departments["Média de Idade"] = df_departments["Departamento"].map(average_age_by_department)
df_departments["Maior Salário"] = df_departments["Departamento"].map(max_salary_by_department)
df_departments["Menor Salário"] = df_departments["Departamento"].map(min_salary_by_department)
df_departments["Soma Salarial"] = df_departments["Departamento"].map(sum_salary_by_department)
df_departments["Media Salarial"] = df_departments["Departamento"].map(average_salary_by_department)
df_departments["Soma Salarial Anual"] = df_departments["Departamento"].map(annual_sum_salary_by_department)
df_departments["Media Salarial Anual"] = df_departments["Departamento"].map(annual_average_salary_by_department)

#Print and save the file
df_departments.to_csv("departments_infos.csv")
df_departments

### Company data

In [ ]:
#Dados da empresa
#Company data
companyName = "Cool Company"
sum_quant_employees = df_employees["ID"].count()
average_age = df_employees["Idade"].mean().round(2)
sum_quant_departments = df_employees["Departamento"].nunique()
sum_salary = df_employees["Salário"].sum().round(2)
average_salary = df_employees["Salário"].mean().round(2)
annual_sum_salary = df_employees["Salário Anual"].sum().round(2)
annual_average_salary = df_employees["Salário Anual"].mean().round(2)

df_company = pd.DataFrame({"Empresa": [companyName]})
df_company["Quantidade de Funcionários"] = sum_quant_employees
df_company["Média de Idade"] = average_age
df_company["Quantidade de Departamentos"] = sum_quant_departments
df_company["Soma Salarial"] = sum_salary
df_company["Media Salarial"] = average_salary
df_company["Soma Salarial Anual"] = annual_sum_salary
df_company["Media Salarial Anual"] = annual_average_salary

df_company.to_csv("company_infos.csv")
df_company